Bước # 1: Phát hiện bài kiểm tra trong một hình ảnh.
Bước # 2: Áp dụng một phép chuyển đổi phối cảnh để trích xuất hình ảnh mắt chim từ trên xuống của bài kiểm tra.
Bước # 3: Trích xuất tập hợp các bong bóng (tức là các lựa chọn câu trả lời có thể có) từ kỳ thi được biến đổi phối cảnh.
Bước # 4: Sắp xếp các câu hỏi / bong bóng thành hàng.
Bước # 5: Xác định câu trả lời được đánh dấu  cho mỗi hàng.
Bước # 6: Tra cứu câu trả lời chính xác trong khóa trả lời  xác định xem có đúng trong lựa chọn hay không.
Bước # 7: Lặp lại tất cả các câu hỏi trong bài kiểm tra.

In [1]:
# các thư viện cần thiết
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
#Đáp án 
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [2]:
# tải ảnh, chuyển ảnh về xám
# làm mịn ảnh và phát hiện cạnh trong ảnh
image = cv2.imread("test.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

In [22]:
cv2.imshow("e",gray)
cv2.waitKey(0)

-1

In [4]:
image[0:5,0:5]=(0,0,255)


In [5]:
cv2.imshow("e",image)
cv2.waitKey(0)

-1

In [6]:
cv2.imshow("edged",edged)
cv2.waitKey(0)

-1

In [7]:
#tìm các đường viền dựa trên bản đồ cạnh
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#khởi tạo đường bao tương ứng với tài liệu
docCnt = None
#phải có ít nhất 1 đường viền được tìm thấy
if len(cnts) > 0:
	# sắp xếp đường viền theo theo thứ tự giảm dần theo diện tích
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
	#chạy vòng lặp qua các đường viền
	for c in cnts:
		#tính chu vi đường viền
		peri = cv2.arcLength(c, True)
		#trả về 1 tập các điểm (x,y) thể hiện 1 đường bao quanh
		approx = cv2.approxPolyDP(c, 0.02 * peri, True)
		# Nếu đường viền có 4 cạnh thì xác định đó là tờ báo
		if len(approx) == 4:
			docCnt = approx
			break

In [8]:
print(docCnt)

[[[131 206]]

 [[119 617]]

 [[448 614]]

 [[430 208]]]


In [9]:
#Biến đổi phối cảnh dựa trên 4 điểm để tạo nên ảnh khi được nhìn trực diện 
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

In [10]:
cv2.imshow("paper",paper)
cv2.imshow("warped",warped)
cv2.waitKey(0)

-1

In [11]:
#đưa ảnh về dạng nhị phân bằng pp ngưỡng otsu
thresh = cv2.threshold(gray, 0, 255,
	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [12]:
thresh.shape
cv2.imshow("a",thresh)
cv2.waitKey(0)

-1

In [13]:
print(thresh)
thresh.shape

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


(700, 525)

In [14]:
cv2.imshow("A",thresh)
cv2.waitKey(0)

-1

In [15]:
thresh[0,229]

255

In [16]:
image[0,0]

array([  0,   0, 255], dtype=uint8)

In [17]:
for i in range(269):
    for j in range(230):
        if thresh[i,j] !=0:
            thresh[i,j]=(0)
            cv2.circle(image,(i, j), 5, (0,0,255), -1)
            break;
    break;


In [18]:
cv2.imshow("p1",image)
cv2.waitKey(0)

-1

In [19]:
# tìm đường viền dựa trên ảnh nhị phân vừa tạo được
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#tạo danh sách đường bao tương ứng với các ô đáp án
questionCnts = []
# chạy vòng lặp đi qua các đường viền
for c in cnts:
	# xác định các thông số của đường bao(tọa độ (x,y), chiều rộng, chiều cao)
	(x, y, w, h) = cv2.boundingRect(c)
	#tính tỉ lệ khung hình của đường bao
	ar = w / float(h)
	# nếu là 1 ô đáp án nó sẽ đủ rộng,đủ cao(20px) và có tỉ lên khung hình gần bằng 1
	if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
		questionCnts.append(c)

In [20]:
questionCnts

[]

In [23]:
# sort the question contours top-to-bottom, then initialize
# the total number of correct answers
questionCnts = contours.sort_contours(questionCnts,
	method="top-to-bottom")[0]
correct = 0
# có 5 câu hỏi và mỗi câu hỏi có 5 câu trả lời
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
	#sắp xếp các câu trả lời theo thứ tự từ trái sang phải
	cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
	#khởi tạo mục câu trả lời đúng
	bubbled = None
	# tạo vòng lặp qua đường viền đã được sắp xếp
	for (j, c) in enumerate(cnts):
		#tạo mặt nạ chỉ hiển thị bong bóng trả lời
		mask = np.zeros(thresh.shape, dtype="uint8")
		cv2.drawContours(mask, [c], -1, 255, -1)
		#áp dụng mặt nạ cho ảnh nhị phân và đếm số pixel khác 0 trong khu vực bong bóng trả lời 
		mask = cv2.bitwise_and(thresh, thresh, mask=mask)
		total = cv2.countNonZero(mask)
		# bong bóng có số lượng pixel khác 0 tối đa là đáp án
		if bubbled is None or total > bubbled[0]:
			bubbled = (total, j)
	# khởi tạo màu đường viền và chỉ số của Câu trả lời chính xác
	color = (0, 0, 255)
	k = ANSWER_KEY[q]
	# kiểm tra câu trả lời có nổi bọt có đúng k
	if k == bubbled[1]:
		color = (0, 255, 0)
		correct += 1
		# vẽ pháC thảo lên bài kiểm tra
	cv2.drawContours(paper, [cnts[k]], -1, color, 3)

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)

[INFO] score: 80.00%


-1